
<h1> DS200A Computer Vision Assignment</h1>

<h2>  Part Three: Classifier training and performance assessment. </h2>	

In [0]:
import pandas as pd
import numpy as np

import warnings 
warnings.filterwarnings('ignore')

In [0]:
train = pd.read_pickle('train.pickle')
test = pd.read_pickle('test.pickle')

In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import StandardScaler
design_matrix = train.drop(columns = ['Pictures', 'Encoding'])
scaler = StandardScaler()
scaler.fit(design_matrix)
design_matrix = scaler.transform(design_matrix)
design_matrix

array([[-0.30054587, -0.05397191, -0.83780732, ..., -0.4273692 ,
         0.3688033 , -0.25769097],
       [-0.28351257, -0.01664309, -0.77696392, ..., -0.51213627,
         0.3688033 , -0.00889756],
       [-0.29585889, -0.03738133, -0.82376653, ..., -0.52628948,
        -0.16958782, -0.14822187],
       ...,
       [ 0.2610962 ,  0.98293987,  0.4679856 , ...,  1.79818506,
        -0.16958782,  0.80714485],
       [-0.07533975,  0.18659162, -0.01876158, ...,  0.08701629,
        -0.16958782,  0.80714485],
       [-0.07607899,  0.31931633, -0.12640759, ...,  0.0950821 ,
        -0.0619096 ,  0.80714485]])

Logistic Regression

In [0]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=42)
parameters = {
    'solver' : ['newton-cg','sag', 'saga', 'lbfgs']
}
lr = GridSearchCV(estimator=lr, param_grid=parameters, cv=5)
lr.fit(design_matrix, train['Encoding']);
accuracies = cross_val_score(lr, design_matrix, train['Encoding'], cv=5);
display(
  accuracies, 
  np.mean(accuracies)
)

array([0.38834951, 0.37171053, 0.35451505, 0.41891892, 0.39249147])

0.3851970955083662

K-nearest Neighbors

In [0]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
parameters = {
    'n_neighbors': np.arange(2, 20), 
    'algorithm': ('auto', 'ball_tree', 'kd_tree', 'brute'),
    'weights': ('uniform', 'distance')
}
knn = GridSearchCV(estimator=knn, param_grid=parameters, cv=5);
knn.fit(design_matrix, train['Encoding']);
accuracies = cross_val_score(knn, design_matrix, train['Encoding'], cv=5);
display(
  accuracies, 
  np.mean(accuracies)
)

array([0.30744337, 0.32565789, 0.33444816, 0.33108108, 0.32764505])

0.32525511064867446

Classification Tree

In [0]:
from sklearn import tree
decision_tree_model = tree.DecisionTreeClassifier(random_state=42)
parameters = {
    'criterion': ('gini', 'entropy'), 
    'splitter': ('best', 'random')
}
decision_tree_model = GridSearchCV(estimator = decision_tree_model, param_grid = parameters, cv=5)
decision_tree_model.fit(design_matrix, train['Encoding'])
accuracies = cross_val_score(decision_tree_model, design_matrix, train['Encoding'], cv=5)
display(
  accuracies, 
  np.mean(accuracies)
)

array([0.27184466, 0.29605263, 0.3277592 , 0.30067568, 0.32764505])

0.30479544319355034

Random Forest

In [0]:
from sklearn.ensemble import RandomForestClassifier
random_forest_model = RandomForestClassifier()
parameters = {
    'n_estimators' : (100, 200, 300), 
    'criterion': ('gini', 'entropy')
}
random_forest_model = GridSearchCV(estimator = random_forest_model, param_grid = parameters, cv=5)
random_forest_model.fit(design_matrix, train['Encoding'])
accuracies = cross_val_score(random_forest_model, design_matrix, train['Encoding'], cv=5)
display(
    accuracies,
    np.mean(accuracies) 
)

array([0.39482201, 0.38157895, 0.38795987, 0.38175676, 0.40273038])

0.38976959044900533

Support Vector Machine

In [0]:
from sklearn.svm import SVC
svc = SVC()
parameters = {
    'kernel': ('linear', 'poly', 'rbf', 'sigmoid'),
    'degree': np.arange(2, 11), 
    'C': np.arange(1, 5)
}
svc = GridSearchCV(estimator = svc, param_grid = parameters, cv=5)
svc.fit(design_matrix, train['Encoding'])
accuracies = cross_val_score(svc, design_matrix, train['Encoding'], cv=5);
display(
  accuracies, 
  np.mean(accuracies)
)

array([0.35598706, 0.36513158, 0.37458194, 0.40540541, 0.40955631])

0.38213245863229206

Final Classifier

In [0]:
final_classifier = random_forest_model
final_classifier = final_classifier.fit(design_matrix, train['Encoding'])

In [0]:
test_design_matrix = test.drop(columns = ['Pictures'])
scaler = StandardScaler()
scaler.fit(test_design_matrix)
test_design_matrix = scaler.transform(test_design_matrix)
test_design_matrix

array([[-0.46682569, -0.8560535 , -0.98421682, ..., -0.48686798,
        -0.12100967, -1.09203153],
       [-0.32338977, -0.32295984, -0.57921406, ..., -0.6017113 ,
        -0.18566857, -2.31983209],
       [-0.22702704, -0.32295984, -0.13537543, ...,  0.12573819,
        -0.18566857,  0.59375812],
       ...,
       [-0.39787013, -0.97945481, -0.3073629 , ..., -0.32631506,
        -0.18566857, -0.18579779],
       [-0.32035434, -0.75733245, -0.09653955, ..., -0.16641281,
        -0.12100967,  0.37938025],
       [-0.40489658, -0.81656508, -0.58476205, ..., -0.3673073 ,
        -0.18566857, -0.14681999]])

In [0]:
test_predictions = final_classifier.predict(test_design_matrix)

classes = ['airplanes', 'bear', 'blimp', 'comet', 'crab', 'dog',
           'dolphin', 'giraffe', 'goat', 'gorilla', 'kangaroo', 
           'killer-whale', 'leopards', 'llama', 'penguin', 'porcupine',
           'teddy-bear', 'triceratops', 'unicorn', 'zebra']

test_predictions = [classes[i] for i in test_predictions]

predictions = pd.DataFrame()
predictions['Predictions'] = test_predictions
predictions.to_csv('predictions.csv')